In [1]:
!pip install simpleitk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

device = "cuda" if torch.cuda.is_available() else "cpu"

Mounted at /content/drive


In [63]:
import importlib
try: 
    importlib.reload(data)
    importlib.reload(networks)
    importlib.reload(trainer)
    print("libs reloaded")
except:
    import data 
    import networks 
    import trainer 
    print("libs imported")
  
Prostate2D = data.Prostate2D
VAE = networks.VAE
VAEGAN = networks.VAEGAN
TrainerVAE = trainer.TrainerVAE
TrainerVAEGAN = trainer.TrainerVAEGAN

libs reloaded


In [83]:
DATA_DIR = '/content/drive/My Drive/Prostate_MRI/'
PROGRESS_DIR_VAE = Path.cwd() / "progress_vae"
PROGRESS_DIR_VAE.mkdir(parents=True, exist_ok=True)

PROGRESS_DIR_VAEGAN = Path.cwd() / "progress_vaegan"
PROGRESS_DIR_VAEGAN.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 32
N_EPOCHS = 200
DECAY_LR_AFTER = 50
LEARNING_RATE = 1e-3
# OPTIMIZER = lambda parameters, lr : torch.optim.Adam(parameters, lr=lr, betas=(0.5,0.999))
OPTIMIZER = lambda parameters, lr : torch.optim.RMSprop(
    parameters,
    lr=lr,
    alpha=0.9,
    eps=1e-8,
    weight_decay=0,
    momentum=0,
    centered=False
)
KLD_ANNEALING_EPOCHS = 25

def lambda_lr(the_epoch):
    return (
        1.0
        if the_epoch < DECAY_LR_AFTER
        else 1 - float(the_epoch - DECAY_LR_AFTER) / (N_EPOCHS - DECAY_LR_AFTER)
    )

In [6]:
indx = np.array([0,2,3,4,5,6,7,8,9,10,11,12])
train_indx, valid_indx = train_test_split(indx, test_size=3, random_state=0)

train_set = Prostate2D(train_indx, data_dir=DATA_DIR)
valid_set = Prostate2D(valid_indx, data_dir=DATA_DIR)

train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
)

valid_loader = DataLoader(
    valid_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False,
)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [108]:
vae_model = VAE(spade=True).to(device)
optimizer = OPTIMIZER(vae_model.parameters(), lr=LEARNING_RATE)
vae_trainer = TrainerVAE(
    vae_model,
    optimizer,
    KLD_ANNEALING_EPOCHS,
    PROGRESS_DIR_VAE,
    train_loader,
    valid_loader,
    device,
)

In [109]:
vae_trainer.train(N_EPOCHS, lambda_lr=lambda_lr)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch #0: Rec_Loss/train = 0.30559, KLD/train = 2.68243 | Rec_Loss/valid = 0.22834, KLD/valid = 1.39153


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch #8: Rec_Loss/train = 0.13496, KLD/train = 0.75688 | Rec_Loss/valid = 0.18437, KLD/valid = 0.64458


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch #9: Rec_Loss/train = 0.12971, KLD/train = 0.67076 | Rec_Loss/valid = 0.18831, KLD/valid = 0.48031


In [84]:
vaegan_model = VAEGAN(spade=True).to(device)
optimizer_enc = OPTIMIZER(vaegan_model.encoder.parameters(), lr=LEARNING_RATE)
optimizer_gen = OPTIMIZER(vaegan_model.generator.parameters(), lr=LEARNING_RATE)
optimizer_disc = OPTIMIZER(vaegan_model.discriminator.parameters(), lr=LEARNING_RATE)
vaegan_trainer = TrainerVAEGAN(
    vaegan_model,
    optimizer_enc,
    optimizer_gen,
    optimizer_disc,
    KLD_ANNEALING_EPOCHS,
    PROGRESS_DIR_VAEGAN,
    train_loader,
    valid_loader,
    1.0,
    device,
)

In [ ]:
torch.autograd.set_detect_anomaly(False)
vaegan_trainer.train(N_EPOCHS, lambda_lr=lambda_lr)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch #0: Rec_Loss/train = 0.47493, KLD/train = 26.62272, Discl_Loss/train = 1.40627, Adv_Loss/train = 1.93376 | Rec_Loss/valid = 0.51763, KLD/valid = 8.45028, Discl_Loss/valid = 1.29692, Adv_Loss/valid = 0.50106


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

Epoch #164: Rec_Loss/train = 0.12855, KLD/train = 0.07191, Discl_Loss/train = 0.34068, Adv_Loss/train = 0.03634 | Rec_Loss/valid = 0.21833, KLD/valid = 0.05215, Discl_Loss/valid = 0.93469, Adv_Loss/valid = 6.26845


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch #165: Rec_Loss/train = 0.12750, KLD/train = 0.07169, Discl_Loss/train = 0.33654, Adv_Loss/train = 0.00635 | Rec_Loss/valid = 0.21974, KLD/valid = 0.05354, Discl_Loss/valid = 0.94505, Adv_Loss/valid = 6.66881


  0%|          | 0/12 [00:00<?, ?it/s]